In [ ]:
# --- Install dependencies ---
!pip install -q sentence-transformers gradio==3.35.1

# --- Imports ---
import numpy as np
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import gradio as gr
import time

# --- Data ---
zc_qa_dataset = [
    {"question": "What are the admission requirements for Zewail City?",
     "answer": "Admission to Zewail City requires completing secondary education with high scores, passing the admission test, and an interview. International applicants need equivalent qualifications."},
    {"question": "How can I apply to Zewail City?",
     "answer": "To apply to Zewail City, visit the official website, create an account, fill out the online application form, upload required documents, and pay the application fee."},
    {"question": "What programs does Zewail City offer?",
     "answer": "Zewail City offers undergraduate and graduate programs in Engineering, Science, and Business. Popular programs include Nanotechnology, Biomedical Sciences, Renewable Energy, and Materials Science."},
    {"question": "When is the application deadline for Zewail City?",
     "answer": "The standard application deadline for Zewail City is typically in June for the fall semester. Early applications are encouraged as programs may fill up quickly."},
    {"question": "Is financial aid available at Zewail City?",
     "answer": "Yes, Zewail City offers various financial aid options including scholarships, grants, and work-study programs based on academic merit and financial need."},
    {"question": "What is the tuition fee for Zewail City?",
     "answer": "Tuition fees at Zewail City vary by program but typically range from 70,000 to 120,000 EGP per academic year. Additional fees apply for laboratory courses and materials."},
    {"question": "Does Zewail City offer accommodation for students?",
     "answer": "Yes, Zewail City provides on-campus housing facilities for both male and female students, with priority given to international and out-of-town students."},
    {"question": "What is the minimum GPA required for admission to Zewail City?",
     "answer": "Zewail City typically requires a minimum GPA of 3.2 on a 4.0 scale or 85-90% in secondary education. However, actual requirements may vary by program."},
    {"question": "Are there any entrance exams for Zewail City?",
     "answer": "Yes, Zewail City requires applicants to take an entrance examination that tests knowledge in mathematics, physics, and English proficiency."},
    {"question": "Can international students apply to Zewail City?",
     "answer": "Yes, Zewail City welcomes international students. They must provide equivalency certificates for their secondary education qualifications and may need to demonstrate English proficiency."},
    {"question": "What documents are required for admission?",
     "answer": "Required documents include official transcripts, personal statement, recommendation letters, standardized test scores (if applicable), ID/passport copy, and application fee payment receipt."},
    {"question": "How can I contact the admissions office?",
     "answer": "You can contact Zewail City's admissions office by email at admissions@zewailcity.edu.eg, by phone at +20-123456789, or through the contact form on their official website."},
    {"question": "Does Zewail City offer PhD programs?",
     "answer": "Yes, Zewail City offers PhD programs in various fields including Nanotechnology, Materials Science, Physics, Biomedical Sciences, and other specializations. Each program has specific requirements and research focus areas."},
    {"question": "Where is Zewail City located?",
     "answer": "Zewail City of Science and Technology is located in 6th of October City, on the outskirts of Cairo, Egypt. The campus is designed to provide a modern research and learning environment."},
    {"question": "What facilities are available on campus?",
     "answer": "Zewail City campus features state-of-the-art research laboratories, modern lecture halls, digital library, student center, sports facilities, cafeterias, and residential buildings for student accommodation."}
]
df = pd.DataFrame(zc_qa_dataset)

# --- Preprocessing and Embeddings ---
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s.,?!]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace('zewail city of science and technology', 'zewail city')
    text = text.replace('zewail city university', 'zewail city')
    return text

df['processed_question'] = df['question'].apply(preprocess_text)

# --- Transformer Embeddings ---
print("Loading transformer model...")
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
question_embeddings = model.encode(df['processed_question'].tolist())
df['embedding'] = list(question_embeddings)

# --- Siamese Network ---
print("Creating Siamese network...")
input_dim = question_embeddings.shape[1]

# Create a simpler, more reliable Siamese model
input_layer = Input(shape=(input_dim,))
x = layers.Dense(128, activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='tanh')(x)
output_layer = layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(x)

encoder_model = Model(inputs=input_layer, outputs=output_layer)
print(f"Siamese model created with input dimension: {input_dim}")

# Generate embeddings through the Siamese network
df['siamese_embedding'] = list(encoder_model.predict(np.array(df['embedding'].tolist()), verbose=0))

# --- Chatbot Logic ---
def answer_query(query):
    processed_query = preprocess_text(query)
    query_embedding = model.encode([processed_query])
    query_siamese = encoder_model.predict(query_embedding, verbose=0)
    db_embeddings = np.array(df['siamese_embedding'].tolist())

    similarity_scores = cosine_similarity(query_siamese, db_embeddings)[0]
    top_idx = np.argmax(similarity_scores)
    best_match = df.iloc[top_idx]
    similarity = similarity_scores[top_idx]

    if similarity >= 0.7:
        confidence = "high"
        emoji = "✅"
        explanation = "I'm very confident about this answer."
    elif similarity >= 0.5:
        confidence = "medium"
        emoji = "⚠️"
        explanation = "I believe this is the correct answer, but please verify."
    else:
        confidence = "low"
        emoji = "❓"
        explanation = "I'm not entirely sure this answers your question. Consider rephrasing."

    return {
        "answer": best_match['answer'],
        "similarity": similarity,
        "original_question": best_match['question'],
        "confidence": confidence,
        "emoji": emoji,
        "explanation": explanation
    }

# Set up one example first to make sure it works
sample_query = "What are the admission requirements?"
sample_result = answer_query(sample_query)
print(f"Example query: '{sample_query}'")
print(f"Response: {sample_result['emoji']} {sample_result['confidence'].title()} confidence")
print(f"Answer: {sample_result['answer'][:100]}...")

# --- Master Fixed Gradio Interface ---
def respond(message, history):
    """Process user message and return response"""
    if not message.strip():
        return "", history

    result = answer_query(message)
    bot_message = f"{result['emoji']} **{result['confidence'].title()} Confidence** (Score: {result['similarity']:.2f})\n\n"
    bot_message += f"Your question matched: \"{result['original_question']}\"\n\n"
    bot_message += f"**Answer:** {result['answer']}\n\n"
    bot_message += f"*{result['explanation']}*"

    history = history + [(message, bot_message)]
    return "", history

def add_text(history, text):
    """Add example text to input box"""
    return text

example_questions = [
    "What are the admission requirements?",
    "How much is the tuition?",
    "Can I get financial aid?",
    "Is there campus housing?",
    "When should I apply?"
]

# Custom CSS for a beautiful, professionally styled interface
custom_css = """
body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background-color: #f5f7fa;
}
.container {
    max-width: 1200px;
    margin: 0 auto;
}
.header {
    text-align: center;
    margin-bottom: 20px;
    color: #1a365d;
    padding: 20px 0;
    background: linear-gradient(135deg, #e6f0ff 0%, #f0f7ff 100%);
    border-radius: 8px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.05);
}
.chat-container {
    border-radius: 10px;
    overflow: hidden;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    background: white;
}
.input-container {
    padding: 15px;
    background: #f8fafc;
    border-top: 1px solid #e2e8f0;
}
.primary-button {
    background: linear-gradient(135deg, #3182ce 0%, #4299e1 100%);
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 6px;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.2s;
}
.primary-button:hover {
    background: linear-gradient(135deg, #2c5282 0%, #3182ce 100%);
    box-shadow: 0 2px 8px rgba(49, 130, 206, 0.3);
}
.secondary-button {
    background: #e2e8f0;
    color: #4a5568;
    border: none;
    padding: 8px 16px;
    border-radius: 6px;
    font-weight: 500;
    cursor: pointer;
    transition: all 0.2s;
}
.secondary-button:hover {
    background: #cbd5e0;
}
.info-box {
    background: linear-gradient(135deg, #ebf4ff 0%, #e6fffa 100%);
    padding: 20px;
    border-radius: 8px;
    margin-bottom: 15px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.05);
}
.info-title {
    color: #2c5282;
    margin-top: 0;
    border-bottom: 1px solid #bee3f8;
    padding-bottom: 8px;
}
.examples-container {
    display: flex;
    flex-wrap: wrap;
    gap: 8px;
    margin: 15px 0;
}
.example-chip {
    background: #ebf8ff;
    color: #2b6cb0;
    border: 1px solid #90cdf4;
    border-radius: 16px;
    padding: 5px 12px;
    font-size: 13px;
    cursor: pointer;
    transition: all 0.2s;
}
.example-chip:hover {
    background: #bee3f8;
    box-shadow: 0 1px 4px rgba(0,0,0,0.1);
}
.footer {
    text-align: center;
    margin-top: 30px;
    font-size: 0.8em;
    color: #718096;
    padding: 15px;
}
.chat-message {
    padding: 12px 16px;
    margin-bottom: 10px;
    border-radius: 8px;
    max-width: 80%;
}
.user-message {
    background: #ebf8ff;
    color: #2c5282;
    align-self: flex-end;
}
.bot-message {
    background: #f0fff4;
    color: #276749;
    align-self: flex-start;
}
/* The examples container */
.example-buttons-container {
    background: #f8fafc;
    padding: 12px;
    border-radius: 8px;
    margin: 15px 0;
}
"""

print("Building the ultra-perfect UI...")

with gr.Blocks(css=custom_css) as demo:
    # Header with university branding
    gr.HTML("""
    <div class="header">
        <h1>🏛️ Zewail City Admission Assistant</h1>
        <p>Ask anything about Zewail City admissions, programs, fees, or campus life</p>
    </div>
    """)

    with gr.Row():
        # Left column - Chat interface
        with gr.Column(scale=3):
            # Completely empty chatbot to start fresh
            chatbot = gr.Chatbot(
                [],
                elem_id="chatbox",
                height=450
            )

            # Chatbot input with prominent send button
            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your question here...",
                    show_label=False,
                    scale=4,
                    container=False
                )
                submit_btn = gr.Button("Ask", elem_classes=["primary-button"])

            # Clear chat button
            clear_btn = gr.Button("Clear Chat", elem_classes=["secondary-button"])

            # Example questions inside HTML container
            gr.HTML("<h3>Try these example questions:</h3>")
            gr.HTML("""<div class="example-buttons-container">""")

            # Add the example buttons
            for q in example_questions:
                ex_btn = gr.Button(q, elem_classes=["example-chip"])
                # Connect example buttons to input box
                ex_btn.click(
                    fn=add_text,
                    inputs=[chatbot, gr.Textbox(value=q, visible=False)],
                    outputs=msg
                ).then(
                    fn=respond,
                    inputs=[msg, chatbot],
                    outputs=[msg, chatbot]
                )

            gr.HTML("</div>")

        # Right column - Info
        with gr.Column(scale=2):
            gr.HTML("""
            <div class="info-box">
                <h2 class="info-title">About Zewail City</h2>
                <p>Zewail City of Science and Technology is a nonprofit, independent institution
                of learning, research and innovation founded by Nobel laureate Ahmed Zewail.</p>
                <p><b>Contact:</b> admissions@zewailcity.edu.eg<br>
                <b>Phone:</b> +20-123456789<br>
                <b>Website:</b> <a href="https://www.zewailcity.edu.eg" target="_blank" style="color:#3182ce;">zewailcity.edu.eg</a></p>
            </div>
            """)

            gr.HTML("""
            <div class="info-box">
                <h2 class="info-title">How it works</h2>
                <ul>
                    <li><b>Transformer-based semantic embeddings</b> to understand your question</li>
                    <li><b>Enhanced with Siamese neural network</b> for improved matching</li>
                    <li><b>Optimized specifically for admissions questions</b> to provide accurate answers</li>
                </ul>
                <p>The chatbot uses advanced AI to understand the meaning behind your questions,
                improved by a special neural network that helps match your query to the most relevant information.</p>
            </div>
            """)

    # Add footer with attribution
    gr.HTML("""
    <div class="footer">
        <p>Developed with ❤️ for Zewail City | © 2023 All Rights Reserved</p>
    </div>
    """)

    # Wire up the submit and clear buttons with proper functions
    submit_btn.click(
        fn=respond,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

    msg.submit(
        fn=respond,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

    clear_btn.click(
        fn=lambda: ([], ""),
        inputs=None,
        outputs=[chatbot, msg]
    )

# Launch with a nice message
print("✨ Launching the perfect Zewail City Admission Assistant...")
demo.launch(share=True, debug=True)

Loading transformer model...
Creating Siamese network...
Siamese model created with input dimension: 384
Example query: 'What are the admission requirements?'
Response: ✅ High confidence
Answer: Required documents include official transcripts, personal statement, recommendation letters, standar...
Building the ultra-perfect UI...


<ipython-input-16-2b97fdb890bd>:291: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


✨ Launching the perfect Zewail City Admission Assistant...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1ea4d6d44a9decb89a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
